# Stereo Fisheye camera calibration for OpenVSLAM

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import sys
import os
from os.path import join
import yaml
import glob
from functools import partial

from enum import Enum
class Camera(Enum):
    OpenCV = 1 # OpenCV
    PySpin = 2 # Spinnaker
    
class Pattern(Enum):
    Chessboard = 1
    Circle = 2

%matplotlib inline

In [ ]:
# User setting

# Input camera
# cam_kind = Camera.OpenCV
cam_kind = Camera.PySpin

# Pattern
pattern_kind = Pattern.Chessboard
# pattern_kind = Pattern.Circle
pattern_one_length = 29.0/1000 # size [m]


# Capture checkboard images
Checkborad pdf file is available at 
 - https://raw.githubusercontent.com/opencv/opencv/master/doc/pattern.png
 - https://raw.githubusercontent.com/opencv/opencv/master/doc/acircles_pattern.png

In [ ]:
# Find pattern
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
if pattern_kind == Pattern.Chessboard:
    pattern_size = (6,9)
    detect_flag = cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE # cv2.CALIB_CB_FAST_CHECK+
    findCorners = partial(cv2.findChessboardCorners, patternSize=pattern_size, flags=detect_flag)
elif pattern_kind == Pattern.Circle:
    pattern_size = (4, 11)
    detect_flag = cv2.CALIB_CB_ASYMMETRIC_GRID+cv2.CALIB_CB_CLUSTERING+cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE
    findCorners = partial(cv2.findCirclesGrid, patternSize=pattern_size, flags=detect_flag)

# Save images if corners are detected
checkBeforeInsert = True


In [ ]:
cam_id = [0, 1]

if cam_kind == Camera.PySpin:
    sys.path.insert(0, './SpinnakerVideoCapture/python')
    from PySpinCap import PySpinManager
    # start manager
    manager = PySpinManager()
    cap = manager.get_multi_camera(cam_id)
if cam_kind == Camera.OpenCV:
    print('Not implemented yet')


In [ ]:
WRITE_VIDEO = False
if WRITE_VIDEO:
    print('#####################')
    print('### Write video')
    elps = []
    save_imgs = []

    for i in range(len(cam_id)):
        cv2.namedWindow('img{}'.format(i), cv2.WINDOW_KEEPRATIO)

    while True:
        start = time.time()

        ret, imgs = cap.read()
        if not ret:
            print('capture error')
            continue

        for i, img in enumerate(imgs):
            img = cv2.resize(img, (640, 640))
            cv2.imshow('img{}'.format(i), img)

        save_imgs.append(imgs)
        key = cv2.waitKey(20)
        elps.append((time.time() - start))
        if key == 27:
                break

        if len(elps) == 100:
            print('- FPS:{}'.format(len(elps) / sum(elps)))
            elps = []

    cv2.destroyAllWindows()
    print('### finish capturing')
    print('#####################')

    # VideoWriter
    save_img_folder = './stereoimg{}'.format(time.strftime("%Y%m%d-%H%M"))
    try:
        os.mkdir(save_img_folder)
    except OSError as exc:
        print(exc)
    print(save_img_folder)

    fourcc = cv2.VideoWriter_fourcc(*"DIVX")
    fps = 18
    height, width, _ = save_imgs[0][0].shape
    writerL = cv2.VideoWriter(join(save_img_folder, 'videoL.avi'), fourcc, fps, (width, height))
    height, width, _ = save_imgs[0][1].shape
    writerR = cv2.VideoWriter(join(save_img_folder, 'videoR.avi'), fourcc, fps, (width, height))

    for it in save_imgs:
        writerL.write(it[0])
        writerR.write(it[1])

    writerL.release()
    writerR.release()

In [ ]:
print('#####################')
print('### start capturing')
elps = []
save_imgs = []

for i in range(len(cam_id)):
    cv2.namedWindow('img{}'.format(i), cv2.WINDOW_KEEPRATIO)
        
while True:
    start = time.time()

    ret, imgs = cap.read()
    if not ret:
        print('capture error')
        continue

    for i, img in enumerate(imgs):
        img = cv2.resize(img, (640, 640))
        cv2.imshow('img{}'.format(i), img)
        
    key = cv2.waitKey(20)
    elps.append((time.time() - start))

    if key == ord('s'):
        cv2.waitKey(300)
        
        ret = False
        if checkBeforeInsert:
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            # Find the chess board corners
            ret, corners = findCorners(gray)
        if ret:
            save_imgs.append(imgs)
            print('save image:', len(save_imgs))
        else:
            print("couldn't find pattern")
    elif key == 27:
        break

    if len(elps) == 100:
        print('- FPS:{}'.format(len(elps) / sum(elps)))
        elps = []

cv2.destroyAllWindows()
print('### finish capturing')
print('#####################')


In [ ]:
# Release everything
cap.release()
if cam_kind == Camera.PySpin:
    manager.release()

In [ ]:
save_img_folder = './stereoimg{}'.format(time.strftime("%Y%m%d-%H%M"))
try:
    os.mkdir(save_img_folder)
except OSError as exc:
    print(exc)
print(save_img_folder)

In [ ]:
# Save images
assert save_imgs[0][0].shape == save_imgs[0][1].shape

for i,it in enumerate(save_imgs):
    print('save image : {:03}'.format(i))
    cv2.imwrite(join(save_img_folder, 'L{:03}.png'.format(i)), it[0])
    cv2.imwrite(join(save_img_folder, 'R{:03}.png'.format(i)), it[1])

# Calib image

## Select pattern type first

In [ ]:
load_folder = 'stereoimg20191101-1622/'
# load_folder = save_img_folder
print('load folder:', load_folder)

In [ ]:
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
%matplotlib inline

# Load images
imageL = []
imageR = []
for fname in sorted(glob.glob('{}/L*.png'.format(load_folder))):
    img = cv2.imread(fname)
    imageL.append(img)
    print(fname)
for fname in sorted(glob.glob('{}/R*.png'.format(load_folder))):
    img = cv2.imread(fname)
    imageR.append(img)
    print(fname)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,10))
ax[0].imshow(imageL[0][:,:,::-1])
ax[1].imshow(imageR[0][:,:,::-1])

In [ ]:
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_CHECK_COND+cv2.fisheye.CALIB_FIX_SKEW
imgsize = None
if pattern_kind == Pattern.Chessboard:
    pattern_size = (6,9)
    objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
    objp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
    objp = objp[:,np.newaxis,:]
    objp *= pattern_one_length
    detect_flag = cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE # cv2.CALIB_CB_FAST_CHECK+
    findCorners = partial(cv2.findChessboardCorners, patternSize=pattern_size, flags=detect_flag)
    
elif pattern_kind == Pattern.Circle:
    pattern_size = (4, 11)
    objp = []
    for i in range(pattern_size[1]):
        for j in range(pattern_size[0]):
            objp.append([2*j+i%2, i ,0])
    objp = np.array(objp, np.float32)[np.newaxis]
    objp *= pattern_one_length
    detect_flag = cv2.CALIB_CB_ASYMMETRIC_GRID+cv2.CALIB_CB_CLUSTERING
    findCorners = partial(cv2.findCirclesGrid, patternSize=pattern_size, flags=detect_flag)


In [ ]:
def get_xy_maxmin(corners, offset=50):
        x_max, y_max = np.max(corners, axis=0)[0]
        x_min, y_min = np.min(corners, axis=0)[0]
        x_max = int(x_max)+offset
        x_min = int(x_min)-offset
        y_max = int(y_max)+offset
        y_min = int(y_min)-offset
        return x_max, x_min, y_max, y_min

In [ ]:
# check first image
if True:
    img = imageL[0].copy()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = findCorners(gray)

    if ret:
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        cv2.drawChessboardCorners(img, pattern_size, corners, ret) 
        x_max, x_min, y_max, y_min = get_xy_maxmin(corners, offset=50)
        fig, ax = plt.subplots(1, 2, figsize=(15,8))
        ax[0].imshow(img[y_min:y_max,x_min:x_max,::-1])
        ax[1].imshow(img[:,:,::-1])
    else:
        print("couldn't find corner")

In [ ]:
def findCornerAll(imgs, prefix=None):
    imgpts = [] # 2d points in image plane.
    imgsize = None
    print("Find corner in all images. Prefix:,", prefix)
    for i, img in enumerate(imgs):
        if imgsize == None:
            imgsize = img.shape[:2]
        else:
            assert imgsize == img.shape[:2], "All images must share the same size."
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # Find the chess board corners
        ret, corners = findCorners(gray)
        # If found, add object points, image points (after refining them)
        if ret == True:
            print('found!{}: imgpts id:{}'.format(i, len(imgpts)))
            cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
            imgpts.append(corners)
            if prefix is not None:
                show = img.copy()
                cv2.drawChessboardCorners(show, pattern_size, corners, ret) 
                fname = join(load_folder, '{}corner{}.jpg'.format(prefix, i))
                cv2.imwrite(fname, show)
        else:
            imgpts.append(None)
            print("couldn't find corner")
            
    return imgpts


In [ ]:
imgLpts = findCornerAll(imageL, prefix='L')
imgRpts = findCornerAll(imageR, prefix='R')

In [ ]:
# Delete if corners are detected in only one image
tmp_id = []
for i, it in enumerate(zip(imgLpts, imgRpts)):
    if it[0] is not None and it[1] is not None:
        tmp_id.append(i)
imgLpts = [imgLpts[i] for i in tmp_id]
imgRpts = [imgRpts[i] for i in tmp_id]
imageL = [imageL[i] for i in tmp_id]
imageR = [imageR[i] for i in tmp_id]
objpts = [objp for i in tmp_id]
print('number of the detected points:', len(tmp_id))

## Check points

In [ ]:
idx = 5
radius = 2
img = imageL[idx].copy()
pt = imgLpts[idx]
_ = [cv2.circle(img, (it[0][0], it[0][1]), radius, [0,255,0]) for it in pt]
x_max, x_min, y_max, y_min = get_xy_maxmin(pt, offset=50)
fig, ax = plt.subplots(1, 2, figsize=(12,12))
ax[0].imshow(img[y_min:y_max,x_min:x_max,::-1])
img = imageR[idx].copy()
pt = imgRpts[idx]
_ = [cv2.circle(img, (it[0][0], it[0][1]), radius, [0,255,0]) for it in pt]
x_max, x_min, y_max, y_min = get_xy_maxmin(pt, offset=50)
ax[1].imshow(img[y_min:y_max,x_min:x_max,::-1])

In [ ]:
def fisheyeCalib(objpoints, imgpoints, imgsize, reject=[]):
    print("#### FISHEYE CALIB START #####")
    N_OK = len(imgpoints)
    print("Found " + str(N_OK) + " valid images for calibration")
    K = np.zeros((3, 3))
    D = np.zeros((4, 1))
    rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
    tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]

    calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC
    calibration_flags |= cv2.fisheye.CALIB_CHECK_COND
    calibration_flags |= cv2.fisheye.CALIB_FIX_SKEW

    #####################
    ## find bad images
    while True:
        try:
            tmp_imgpoints = [imgpoints[i] for i in range(len(imgpoints)) if not i in reject]
            tmp_objpoints = [objpoints[i] for i in range(len(imgpoints)) if not i in reject]
            rms, _, _, _, _ = \
                cv2.fisheye.calibrate(
                    tmp_objpoints, tmp_imgpoints, imgsize[::-1], K, D, rvecs, tvecs,
                    calibration_flags, (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
                )
            print('Current rms:', rms)
            break
        except Exception as e:
#             print("type error: " + str(e))
            msg = str(e).split('input array ')[1]
            reject_idx = int(msg.split()[0])
            reject.append(reject_idx+len(reject))
            print('reject_idx:', reject[-1])

    ######################
    ## final refinement
    calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC
    calibration_flags |= cv2.fisheye.CALIB_CHECK_COND
    calibration_flags |= cv2.fisheye.CALIB_FIX_SKEW
    calibration_flags |= cv2.fisheye.CALIB_USE_INTRINSIC_GUESS
    
    # use rejected point or not
    if True:
        tmp_imgpoints = [imgpoints[i] for i in range(len(imgpoints)) if not i in reject]
        tmp_objpoints = [objpoints[i] for i in range(len(imgpoints)) if not i in reject]
    else:
        tmp_imgpoints = imgpoints
        tmp_objpoints = objpoints

    rms, K, D, rvecs, tvecs = \
        cv2.fisheye.calibrate(
            tmp_objpoints, tmp_imgpoints, imgsize[::-1], K, D, rvecs, tvecs,
            calibration_flags, (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
        )

    print('final result')
    print("K=np.array(" + str(K.tolist()) + ")")
    print("D=np.array(" + str(D.tolist()) + ")")
    print("RMS:", rms)
    print("#### FISHEYE CALIB END #####")
    return K, D, rvecs, tvecs, rms

## Calibration

In [ ]:
imgsize = imageL[0].shape[:2]
print('Left calib:')
K1, D1, rvecs, tvecs, rms = fisheyeCalib(objpts, imgLpts, imgsize)
print('Right calib:')
K2, D2, rvecs, tvecs, rms = fisheyeCalib(objpts, imgRpts, imgsize)

In [ ]:
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC
calibration_flags |= cv2.fisheye.CALIB_CHECK_COND
calibration_flags |= cv2.fisheye.CALIB_FIX_SKEW
calibration_flags |= cv2.fisheye.CALIB_USE_INTRINSIC_GUESS

ret, K1, D1, K2, D2, R, T = cv2.fisheye.stereoCalibrate(
    objpts, imgLpts, imgRpts, K1, D1, K2, D2, imgsize, 
    flags=calibration_flags, criteria=(cv2.TERM_CRITERIA_COUNT+cv2.TERM_CRITERIA_EPS, 30, 1e-6))
print("K1=np.array(" + str(K1.tolist()) + ")")
print("D1=np.array(" + str(D1.tolist()) + ")")
print("K2=np.array(" + str(K2.tolist()) + ")")
print("D2=np.array(" + str(D2.tolist()) + ")")
print("R:",R)
print("T:",T)
print("RMS:", ret)

In [ ]:
# Save calibration results
fname  = join(load_folder,'calib_stereofisheye_results.yml')

fs = cv2.FileStorage(fname, cv2.FILE_STORAGE_WRITE)
fs.write('K1',K1)
fs.write('D1',D1)
fs.write('K2',K2)
fs.write('D2',D2)
fs.write('R',R)
fs.write('T',T)
fs.write('imgsize', imgsize)
fs.release()

In [ ]:
# Load calib
if not 'K1' in locals(): 
    load_folder = './stereoimg20191030-1734/'
    print('Need to load calibration data:')
    print('Loading from ', load_folder)
    fname  = join(load_folder,'calib_stereofisheye_results.yml')
    fs = cv2.FileStorage(fname, cv2.FILE_STORAGE_READ)
    print(fs.root().keys())
    K1 = fs.getNode('K1').mat()
    D1 = fs.getNode('D1').mat()
    K2 = fs.getNode('K2').mat()
    D2 = fs.getNode('D2').mat()
    R = fs.getNode('R').mat()
    T = fs.getNode('T').mat()
    imgsize = fs.getNode('imgsize').mat().flatten()
    imgsize = (int(imgsize[0]), int(imgsize[1]))
    fs.release()
    print('Finish loading')

## Rectify stereo pair

In [ ]:
# R1, R2, P1, P2, Q = cv2.fisheye.stereoRectify(
#     K1, D1, K2, D2, imgsize, R, T, 
#     cv2.CALIB_ZERO_DISPARITY, balance=1.0, fov_scale=0.01)

In [ ]:
from numpy.linalg import norm
# https://github.com/opencv/opencv/blob/master/modules/calib3d/src/fisheye.cpp
def stereoRectify(R, T):
    rvec, _ = cv2.Rodrigues(R)
    rvec *= -0.5 # get average rotation
    r_r, _ = cv2.Rodrigues(rvec) # rotate cameras to same orientation by averaging
    t = r_r.dot(T)

    if t[0] > 0:
        uu = np.array([1.0, 0, 0])[:,np.newaxis]
    else:
        uu = np.array([-1.0, 0, 0])[:,np.newaxis]

    # calculate global Z rotation
    ww = np.cross(t.flatten(), uu.flatten())
    nw = norm(ww)
    if nw > 0.0:
        ww *= np.arccos(np.abs(t[0])/norm(t))/nw
    wr, _ = cv2.Rodrigues(ww)
    rectR1 = wr.dot(r_r.T)
    rectR2 = wr.dot(r_r)
    rectT  = rectR2.dot(T)
    return rectR1, rectR2, rectT

def stereoNewK(K1, K2, scale=1.0):
    newK = np.eye(3)
    fc_new = min(K1[1,1], K2[1,1])
    cc_new = (K1[:2,2]+K2[:2,2])*0.5
    newK[0,0] = fc_new*scale
    newK[1,1] = fc_new*scale
    newK[:2,2] = cc_new   
    return  newK

In [ ]:
rectR1, rectR2, rectT = stereoRectify(R, T)

# New camera matrix
newK = stereoNewK(K1, K2)
newK

In [ ]:
idx = 10
radius = 3
thickness = 3
fig, ax = plt.subplots(2, 2, figsize=(12,12))

# Viualize rectify points
# Image1
img = imageL[idx].copy()
img_size = (img.shape[1], img.shape[0])
pts = imgLpts[idx]
radius = 3
_ = [cv2.circle(img, (it[0][0], it[0][1]), radius, [0,255,0], thickness) for it in pts]

ax[0][0].imshow(img)
ax[0][0].set_title('Left image')

img = imageL[idx].copy()
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K1, D1, rectR1, newK, img_size, cv2.CV_16SC2) 
rectify = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

undistort_pts = cv2.fisheye.undistortPoints(pts, K1, D1, R=rectR1, P=newK)
_ = [cv2.circle(rectify, (it[0][0], it[0][1]), radius, [0,255,0], thickness) for it in undistort_pts]
ax[0][1].imshow(rectify[:,:,::-1])

# Image 2
img = imageR[idx].copy()
pts = imgRpts[idx]

_ = [cv2.circle(img, (it[0][0], it[0][1]), radius, [0,255,0], thickness) for it in pts]
ax[1][0].imshow(img)
ax[1][0].set_title('Right image')

img = imageR[idx].copy()
pts = imgRpts[idx]
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K2, D2, rectR2, newK, img_size, cv2.CV_16SC2) 
rectify = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

undistort_pts = cv2.fisheye.undistortPoints(pts, K2, D2, R=rectR2, P=newK)
_ = [cv2.circle(rectify, (it[0][0], it[0][1]), radius, [0,255,0], thickness) for it in undistort_pts]
ax[1][1].imshow(rectify[:,:,::-1])

## Check epipolar

In [ ]:
total_line = pattern_size[0]
pts = imgLpts[idx]

undistort_pts = cv2.fisheye.undistortPoints(pts, K1, D1, R=rectR1, P=newK)
# lines = np.linspace(0, imgsize[1], total_line).astype(np.int)[1:-1]
lines = undistort_pts[:,:,1].flatten()[:total_line]

colors = [tuple(np.random.randint(0,255,3).tolist()) for _ in range(total_line)]

img = imageL[idx]
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K1, D1, rectR1, newK, img_size, cv2.CV_16SC2) 
rectify = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
for i, l in enumerate(lines):
    rectify = cv2.line(rectify, (0, l),(imgsize[0], l), colors[i], thickness)
fig, ax = plt.subplots(1,2,figsize=(12,12))
ax[0].imshow(rectify[:,:,::-1])
img = imageR[idx]
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K2, D2, rectR2, newK, img_size, cv2.CV_16SC2) 
rectify = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
for i, l in enumerate(lines):
    rectify = cv2.line(rectify, (0, l),(imgsize[0], l), colors[i], thickness)
ax[1].imshow(rectify[:,:,::-1])

# Check epipolar
pts = imgLpts[idx]
undist_Lpts = cv2.fisheye.undistortPoints(pts, K1, D1, R=rectR1, P=newK)
pts = imgRpts[idx]
undist_Rpts = cv2.fisheye.undistortPoints(pts, K2, D2, R=rectR2, P=newK)

print('Average epopolar error:', np.abs(undist_Lpts[:,:,1] - undist_Rpts[:,:,1]).mean())

## Triangulate points

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# %matplotlib qt  # wx, gtk, osx, tk, empty uses default
%matplotlib notebook

In [ ]:
# Projection matrix
P1 = np.hstack([newK, np.zeros((3, 1))])
fT2 = np.array([newK[0,0] * rectT[0,0], 0., 0.])[np.newaxis].T
P2 = np.hstack([newK, fT2])
pt3d = cv2.triangulatePoints(P1, P2, undist_Lpts, undist_Rpts)
pt3d = pt3d[:3]/pt3d[3] # homogeneous to euclidean

# Camera pose
cam = np.zeros((3, 2))
cam[0, 1] = -rectT[0, 0]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111 , projection='3d')
sc = ax.scatter(pt3d[0], pt3d[1], pt3d[2],marker='o') 
sc = ax.scatter(cam[0], cam[1], cam[2],marker='x')

# show axis
axis_scale = cam[0, 1]/2
ax.plot([0, axis_scale], [0, 0], [0, 0], 'r')
ax.plot([0, 0], [0, axis_scale], [0, 0], 'g')
ax.plot([0, 0], [0, 0], [0, axis_scale],  'b')

# scale equal
tmppts = np.hstack([pt3d, cam])
max_range = (np.max(tmppts, axis=1)-np.min(tmppts, axis=1)).max()/2
mid = (np.max(tmppts, axis=1)+np.min(tmppts, axis=1))/2
ax.set_xlim(mid[0]-max_range, mid[0]+max_range)
ax.set_ylim(mid[1]-max_range, mid[1]+max_range)
ax.set_zlim(mid[2]-max_range, mid[2]+max_range)
plt.show()

In [ ]:
# Triangulate points from scratch
# http://cmp.felk.cvut.cz/cmp/courses/TDV/2012W/lectures/tdv-2012-07-anot.pdf
pt_idx = 0
results = []
for pt_idx in range(undist_Lpts.shape[0]):
    u1, v1 = undist_Lpts[pt_idx][0]
    u2, v2 = undist_Rpts[pt_idx][0]
    A = np.vstack(
        [u1*P1[2]-P1[0], v1*P1[2]-P1[1],
         u2*P2[2]-P2[0], v2*P2[2]-P2[1]])
    
    u, s, v = np.linalg.svd(A)
    # u.dot(np.diag(s)).dot(v)
    results.append(v[-1][:,np.newaxis])
results = np.hstack(results)
results = results[:3]/results[3]
np.allclose(pt3d, results)

In [ ]:
def printForOpenVSLAM(K, D, img, fps=0, load_folder=load_folder, filename='calib_fisheye_results.txt'):
    data = {}
    data['fx'] = K[0][0]
    data['fy'] = K[1][1]
    data['cx'] = K[0][2]
    data['cy'] = K[1][2]

    data['k1'] = D[0][0]
    data['k2'] = D[1][0]
    data['k3'] = D[2][0]
    data['k4'] = D[3][0]
    
    data['fps'] = fps
    data['cols'] = img.shape[1]
    data['rows'] = img.shape[0]
    
    print_array=[]
    for it in ['fx', 'fy', 'cx', 'cy']:
        print_array.append('Camera.{}: {}'.format(it, data[it]))
    print_array.append('')
    for it in ['k1', 'k2', 'k3', 'k4']:
        print_array.append('Camera.{}: {}'.format(it, data[it]))
    print_array.append('')
    for it in ['fps', 'cols', 'rows']:
        print_array.append('Camera.{}: {}'.format(it, data[it]))

    with open(join(load_folder,filename), mode='w') as f:
        for it in print_array:
            print(it)
            print(it, file=f)

In [ ]:
print("Calibration result L")
printForOpenVSLAM(K1, D1, imageL[0], fps=30, load_folder=load_folder, filename='calib_fisheyeL_results.txt')
print("Calibration result R")
printForOpenVSLAM(K2, D2, imageR[0], fps=30, load_folder=load_folder, filename='calib_fisheyeR_results.txt')

# Rectify

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 8))
idx = 8
img = imageL[idx].copy()
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K1, D1, rectR1, newK, img_size, cv2.CV_16SC2) 
rectify = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
ax[0].imshow(rectify)
ax[0].set_title('Left image')

# Image 2
img = imageR[idx].copy()
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K2, D2, rectR2, newK, img_size, cv2.CV_16SC2) 
rectify = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
ax[1].imshow(rectify)
ax[1].set_title('Right image')

# Rectify images
## Define map first
e.g.,  
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), newK, img_size, cv2.CV_16SC2) 

In [ ]:
# Input camera
# cam_kind = Camera.cv2
cam_kind = Camera.PySpin

cam_id = [0, 1]

if cam_kind == Camera.PySpin:
    sys.path.insert(0, './SpinnakerVideoCapture/python')
    from PySpinCap import PySpinManager
    # start manager
    manager = PySpinManager()
    cap = manager.get_multi_camera(cam_id)
if cam_kind == Camera.OpenCV:
    print('Not implemented yet')



In [ ]:
print('#####################')
print('### start capturing')

mapL1, mapL2 = cv2.fisheye.initUndistortRectifyMap(K1, D1, rectR1, newK, img_size, cv2.CV_16SC2) 
mapR1, mapR2 = cv2.fisheye.initUndistortRectifyMap(K2, D2, rectR2, newK, img_size, cv2.CV_16SC2) 
remap = [partial(cv2.remap, map1=mapL1,map2=mapL2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT),
         partial(cv2.remap, map1=mapR1,map2=mapR2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)]    


elps = []
save_imgs = []
for i, _ in enumerate(remap):
    cv2.namedWindow('img{}'.format(i), cv2.WINDOW_KEEPRATIO)

while True:
    start = time.time()

    ret, imgs = cap.read()
    if not ret:
        print('capture error')
        continue

    for i, img in enumerate(imgs):
        rectify = remap[i](img)
        rectify = cv2.resize(rectify, (640, 640))
        cv2.imshow('img{}'.format(i), rectify)

    key = cv2.waitKey(20)
    elps.append((time.time() - start))

    if key == ord('s'):
        cv2.waitKey(300)
        save_imgs.append(img)
    elif key == 27:
        break

    if len(elps) == 100:
        print('- FPS:{}'.format(len(elps) / sum(elps)))
        elps = []

cv2.destroyAllWindows()
print('### finish capturing')
print('#####################')
